In [2]:
import sqlite3
conn = sqlite3.connect("employees.db")
cursor = conn.cursor()

# List all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables:", tables)

# Preview data (replace `your_table_name` with the actual name)
cursor.execute("SELECT salary FROM employees WHERE name = 'Alice';")
rows = cursor.fetchall()
print("Sample data:", rows)

conn.close()

Tables: [('employees',)]
Sample data: [(62000,)]


In [8]:
from langchain.llms import Ollama
from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain.prompts import PromptTemplate
from typing import TypedDict, Optional

# -----------------------
# Define State
# -----------------------
class EmployeeState(TypedDict):
    query: str              # raw user question
    name: Optional[str]     # extracted employee name
    salary: Optional[int]   # salary if found
    result: Optional[str]   # final message

# -----------------------
# Setup Ollama LLM
# -----------------------
llm = Ollama(model="gemma3:4b")

prompt = PromptTemplate(
    input_variables=["query"],
    template="Extract the employee name from this query: {query}. "
             "Just return the name, nothing else."
             "If no name can be found, return exactly None (without quotes). "
             "Do not include any other text."
)

# Connect to SQLite database
def query_employee_db(sql_query: str) -> str:
    try:
        conn = sqlite3.connect("employees.db")
        cursor = conn.cursor()
        cursor.execute(sql_query)
        results = cursor.fetchall()
        conn.close()
        #return str(results)
        return results
    except Exception as e:
        return f"Database error: {e}"

# -----------------------
# Define Nodes
# -----------------------
def extract_name_node(state: EmployeeState) -> EmployeeState:
    print("➡️ Enter extract_name_node, state =", state)
    """Use Ollama to extract employee name from natural query"""
    name = llm.invoke(prompt.format(query=state["query"])).strip()
    if name.lower() == "none":
        state["name"] = None
    else:
        state["name"] = name    
    print("➡️ Exit extract_name_node, state =", state)
    return state

def name_not_found_node(state: EmployeeState) -> EmployeeState:
    print("➡️ Enter name_not_found_node, state =", state)
    if state["name"] is None:
        state["result"] = f"Sorry, i couldnt extract name from the query, did u froget to put a name?"
    print("➡️ Exit name_not_found_node, state =", state)
    return state

def find_salary_node(state: EmployeeState) -> EmployeeState:
    print("➡️ Enter find_salary_node, state =", state)
    # cursor.execute("SELECT salary FROM employees WHERE name=?", (state["name"],))
    sql = f"SELECT salary FROM employees WHERE name='{state['name']}'"
    row = query_employee_db(sql)
    if row and row[0] is not None:
        state["salary"] = row[0]
        state["result"] = f"Salary of {state['name']} is {row[0]}"
    else:
        state["salary"] = None
    print("➡️ Exit find_salary_node, state =", state)
    return state

def default_salary_node(state: EmployeeState) -> EmployeeState:
    print("➡️ Enter default_salary_node, state =", state)
    if state["salary"] is None:
        state["salary"] = 50000
        state["result"] = f"Default salary for {state['name']} is 50000"
    print("➡️ Exit default_salary_node, state =", state)
    return state

# -----------------------
# Step 6. Build LangGraph
# -----------------------
graph = StateGraph(EmployeeState)

graph.add_node("extract_name", extract_name_node)
graph.add_node("name_not_found", name_not_found_node)
graph.add_node("find_salary", find_salary_node)
graph.add_node("default_salary", default_salary_node)

graph.set_entry_point("extract_name")
#graph.add_edge("extract_name", "find_salary")

# Conditional edge: if salary not found → go to default_salary
def salary_not_found(state: EmployeeState) -> str:
    return "default_salary" if state["salary"] is None else END

def name_not_exists(state: EmployeeState) -> str:
    return "name_not_found" if state["name"] is None else  "find_salary"

graph.add_conditional_edges(
    "extract_name",
    name_not_exists,
    {"name_not_found": "name_not_found", "find_salary": "find_salary"}
)

graph.add_conditional_edges(
    "find_salary",
    salary_not_found,
    {"default_salary": "default_salary", END: END}
)


graph.add_edge("default_salary", END)
graph.add_edge("name_not_found", END)

app = graph.compile()
app.get_graph().print_ascii()
# -----------------------
# Step 7. Run Queries
# -----------------------
result = app.invoke({"query": "salary is how much?"})
print(result["result"])   # Salary of Alice is 80000

#result = app.invoke({"query": "What is Charlie getting paid?"})
#print(result["result"])   # Default salary for Charlie is 50000


                              +-----------+                                
                              | __start__ |                                
                              +-----------+                                
                                     *                                     
                                     *                                     
                                     *                                     
                             +--------------+                              
                             | extract_name |                              
                             +--------------+....                          
                             ...                 .....                     
                           ..                         .....                
                         ..                                ....            
                +-------------+                                ...         
            